### Este documento es una versión editada del documento original:
    https://github.com/gcgus/Kobologia/blob/main/code/Koblog%C3%ADa_Bucles.ipynb
### Creado por Gustavo Castelo en 2022.
### Editado por Bosco Olives en 2023.

In [ ]:
import os
import numpy as np
import librosa
from scipy.io import wavfile

# Libreta utilizada para la creción de bucles a partir de muestras de sonido

In [ ]:
path = 'path/to/audios'


In [ ]:
dir = "cut_synthesized" #carpeta en que se crearan los nuevos audios

if not os.path.exists(dir):
    os.mkdir(dir)

for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    if os.path.isdir(folder_path):
        cut_folder_path = os.path.join(dir, folder)
        if not os.path.exists(cut_folder_path): #si no existe la carpeta dir se crea una con ese nombre
            os.mkdir(cut_folder_path)
        
        for f in os.listdir(folder_path):
            file_path = os.path.join(folder_path, f)
            
            data, sr = librosa.load(file_path, sr=48000) #substituir con el sampleRate adecuado
            
            init = 0.5
            end = 0.9
            dur = end - init
            data_trim = data[int(init * sr):int(end * sr)]
            
            zero_crosses = np.nonzero(np.diff(data_trim > 0))[0]
            
            if len(zero_crosses) > 0:
                minr = 100000
                mini = 0
                wind = 10

                for i in range(len(zero_crosses)):
                    if data_trim[zero_crosses[i] + 1] > 0:
                        zero_crosses = zero_crosses[i:]
                        break

                for i in range(1, len(zero_crosses)):
                    if zero_crosses[i] + wind > len(data_trim):
                        break

                    rest = np.mean(np.abs(data_trim[zero_crosses[0]:zero_crosses[0] + wind] - data_trim[zero_crosses[i]:zero_crosses[i] + wind]))

                    if rest < minr:
                        minr = rest
                        mini = i

                data_zero = data_trim[zero_crosses[0]:zero_crosses[mini]]
                wdur = 1
                loop = np.tile(data_zero, round((wdur * sr) / len(data_zero)))

                output_file_path = os.path.join(cut_folder_path, f)
                wavfile.write(output_file_path, sr, data_zero)
            else:
                print("No se encontraron cruces por cero en el archivo:", file_path)
